Ottenimento data nascita artisti_DEFINITIVO con ottimizzazioni: Varianti per trovare la pagina Wikipedia ((cantante), parole chiave, pseudonimi)

Parsing avanzato della data di nascita con:

(Luogo, data)

nata/o a [luogo] nel [anno]

nato/nata il [giorno mese anno]

Gestione di nomi composti o pseudonimi

Solo "Maschio" e "Femmina" vengono ricercati, tutti gli altri restano con "Da verificare" nel file finale

In [3]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm

# Impostazioni Wikipedia
wikipedia.set_lang("it")
wikipedia.set_rate_limiting(True)

# Carica il file Excel
file_path = "File Unico Sanremo.xlsx"
df = pd.read_excel(file_path, sheet_name="Artisti Unici con genere")

# Divide artisti individuali e gruppi
df_individuali = df[df["Genere"].isin(["Maschio", "Femmina"])].copy()
df_gruppi = df[~df["Genere"].isin(["Maschio", "Femmina"])].copy()
df_gruppi["Data di nascita"] = "Da verificare"

# Cache e dizionario mesi
dob_cache = {}
mesi_ita = {
    "gennaio": "01", "febbraio": "02", "marzo": "03", "aprile": "04",
    "maggio": "05", "giugno": "06", "luglio": "07", "agosto": "08",
    "settembre": "09", "ottobre": "10", "novembre": "11", "dicembre": "12"
}

# Trova la pagina Wikipedia provando varie alternative
def cerca_pagina(artista):
    varianti = [
        artista,
        f"{artista} (cantante)",
        f"{artista} cantante",
        f"{artista} musicista",
        f"{artista} cantautore",
        f"{artista} cantautrice",
        f"{artista} sanremo"
    ]

    for nome in varianti:
        try:
            return wikipedia.page(nome, auto_suggest=False, preload=False)
        except (wikipedia.exceptions.DisambiguationError, wikipedia.exceptions.PageError):
            continue
        except Exception:
            continue
    return None

# Estrae una data da una stringa (diversi formati supportati)
def estrai_data_da_testo(testo):
    testo = testo.lower()

    # Caso: (luogo, 29 settembre 1997)
    match = re.search(r'\(([^,]+),\s*(\d{1,2})\s+([a-z]+)\s+(\d{4})\)', testo)
    if match:
        giorno = match.group(2).zfill(2)
        mese = mesi_ita.get(match.group(3))
        anno = match.group(4)
        if mese:
            return f"{giorno}/{mese}/{anno}"

    # Caso: nato/nata il 3 aprile 1980
    match = re.search(r'(nato|nata) il (\d{1,2})\s+([a-z]+)\s+(\d{4})', testo)
    if match:
        giorno = match.group(2).zfill(2)
        mese = mesi_ita.get(match.group(3))
        anno = match.group(4)
        if mese:
            return f"{giorno}/{mese}/{anno}"

    # Caso: nata/o a X nel 2002
    match = re.search(r'(nato|nata) a [a-zàèéòùì\s]+ nel (\d{4})', testo)
    if match:
        anno = match.group(2)
        return f"01/01/{anno}"  # Data generica, ma utile

    return None

# Estrazione completa data di nascita
def estrai_data_nascita(artista):
    if artista in dob_cache:
        return dob_cache[artista]

    # Salta nomi multipli
    if any(sep in artista.lower() for sep in [",", " e ", " - "]):
        print(f"⚠️ Nomi multipli rilevati, salto: {artista}")
        return "Da verificare"

    print(f"🟨 Elaborazione: {artista}")
    pagina = cerca_pagina(artista)
    if pagina is None:
        print(f"❌ Nessuna pagina trovata per: {artista}")
        return "Da verificare"

    contenuto = (pagina.summary + " " + pagina.content).lower()

    # Se c'è un "pseudonimo di", esamina anche il nome reale
    if "pseudonimo di" in contenuto:
        match_nome = re.search(r"pseudonimo di ([a-z\s']+)", contenuto)
        if match_nome:
            vero_nome = match_nome.group(1).strip()
            print(f"🔍 Pseudonimo rilevato, provo con vero nome: {vero_nome}")
            pagina_vero_nome = cerca_pagina(vero_nome)
            if pagina_vero_nome:
                contenuto += " " + pagina_vero_nome.summary.lower()

    data = estrai_data_da_testo(contenuto)
    if data:
        dob_cache[artista] = data
        return data

    print(f"⚠️ Nessuna data trovata per: {artista}")
    return "Da verificare"

# Applica la funzione in parallelo
def apply_parallel(df, func, descrizione):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, artista): artista for artista in df["Artista"]}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            artista = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {artista} — {e}")
                results.append("Errore")
    return results

# Applica solo ai cantanti singoli
df_individuali["Data di nascita"] = apply_parallel(df_individuali, estrai_data_nascita, "Estrazione data individuale")

# Ricombina il dataframe
df_finale = pd.concat([df_individuali, df_gruppi]).sort_index()

# Salvataggio finale
output_file = "Artisti_con_data_nascita_ottimizzata.xlsx"
df_finale.to_excel(output_file, index=False)

print(f"\n✅ File salvato con successo: {output_file}")


🟨 Elaborazione: Achille Togliani
🟨 Elaborazione: Oscar Carboni
🟨 Elaborazione: Gino Latilla
🟨 Elaborazione: Zucchero
🟨 Elaborazione: Will
🟨 Elaborazione: Virginio
🟨 Elaborazione: Ultimo
🟨 Elaborazione: Tricarico
🟨 Elaborazione: Tiziano Orecchio
🟨 Elaborazione: Stefano Tosi


Estrazione data individuale:   0%|          | 0/470 [00:00<?, ?it/s]D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\bocci\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Estrazione data individuale:   0%|          | 2/470 [00:01<05:28,  1.43it/s]

⚠️ Nessuna data trovata per: Gino Latilla
🟨 Elaborazione: Ugo Molinari
⚠️ Nessuna data trovata per: Virginio
🟨 Elaborazione: Gianni Marzocchi
⚠️ Nessuna data trovata per: Tricarico
🟨 Elaborazione: Stefano Rosso
⚠️ Nessuna data trovata per: Achille Togliani
🟨 Elaborazione: Silvano Vittorio
⚠️ Nessuna data trovata per: Oscar Carboni
🟨 Elaborazione: Sangiovanni


Estrazione data individuale:   1%|▏         | 6/470 [00:02<02:08,  3.62it/s]

⚠️ Nessuna data trovata per: Zucchero
🟨 Elaborazione: Samuel


...

🔍 Pseudonimo rilevato, provo con vero nome: sara mattei
🔍 Pseudonimo rilevato, provo con vero nome: carla bissi


Estrazione data individuale:  99%|█████████▉| 466/470 [02:13<00:01,  2.75it/s]

🔍 Pseudonimo rilevato, provo con vero nome: gaia gozzi


Estrazione data individuale: 100%|██████████| 470/470 [02:16<00:00,  3.45it/s]


✅ File salvato con successo: Artisti_con_data_nascita_ottimizzata.xlsx
